<div style="text-align:center; color:  #13476a ; font-size:20px">

<h1> Modelado de temas  </h1> 

Alejandro Fandiño B
    
</div>

<h5> </h5>

- Hacer pre-procesamiento del texto

- Modelo de LDA

- Visualización de LDA

In [1]:
import re
import pandas as pd 
from pprint import pprint

from nltk.corpus import stopwords
stopwords = stopwords.words('spanish')

import pyLDAvis.gensim
from gensim.models import LdaModel
from gensim.corpora import Dictionary

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

C:\Users\ALEJANDRO\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
path = 'reviews_vidjew_es.csv'
data = pd.read_csv(path)
data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games


# Punto 1 : Hacer pre-procesamiento del texto

In [3]:
def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = [palabra for palabra in texto.split() if palabra not in stopwords]
    return texto

In [5]:

data['pp'] = data['review_body'].apply(lambda texto: pre_procesado(texto))

data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,pp
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry,"[buen, buena, calidad, buena, presentación]"
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games,"[producto, perfecto, salir, casa, nintendo, sw..."
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games,"[funciona, nintendo, switch, forma, emparejarl..."
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games,"[recomendado, utilizo, pc, dan, ningún, proble..."
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games,"[cable, funciona, bien, podria, ser, mas, larg..."


# Punto 2

In [8]:
# Representación de los documentos en forma de diccionario
dictionary = Dictionary(data['pp'].values)
dictionary

In [11]:
# Filtrar palabras muy frecuentes o infrecuentes
dictionary.filter_extremes(no_below=5, no_above=0.5)

corpus = [dictionary.doc2bow(text) for text in data['pp'].values]


In [12]:
# Train the topic model
model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=7, passes=4)


In [13]:
model.print_topics(num_words=10)

[(0,
  '0.047*"calidad" + 0.030*"precio" + 0.023*"si" + 0.022*"producto" + 0.021*"regalo" + 0.017*"buena" + 0.016*"bien" + 0.014*"bonito" + 0.014*"plata" + 0.013*"pulsera"'),
 (1,
  '0.029*"calidad" + 0.026*"juego" + 0.023*"bien" + 0.021*"ps" + 0.018*"foto" + 0.015*"pedido" + 0.015*"precio" + 0.014*"mas" + 0.013*"jugar" + 0.013*"buena"'),
 (2,
  '0.032*"precio" + 0.025*"producto" + 0.022*"calidad" + 0.019*"bonitos" + 0.018*"compra" + 0.016*"cadena" + 0.014*"foto" + 0.014*"aunque" + 0.013*"buen" + 0.013*"colgante"'),
 (3,
  '0.039*"llegado" + 0.029*"bonito" + 0.022*"día" + 0.016*"bien" + 0.015*"calidad" + 0.013*"amazon" + 0.013*"todavía" + 0.013*"buena" + 0.013*"entrega" + 0.010*"si"'),
 (4,
  '0.032*"bien" + 0.018*"juego" + 0.018*"parece" + 0.018*"calidad" + 0.016*"bastante" + 0.016*"precio" + 0.014*"buena" + 0.013*"pena" + 0.013*"esperaba" + 0.013*"recomiendo"'),
 (5,
  '0.048*"bien" + 0.026*"si" + 0.021*"juego" + 0.015*"llegó" + 0.015*"perfecto" + 0.015*"funciona" + 0.014*"mando" + 0

# Punto 3

In [14]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(model, corpus, dictionary, sort_topics=True)
pyLDAvis.display(lda_display)